Instalações de bibliotecas necessárias:

OBS: A numpy possui um erro na biblioteca boruta_py, com isso é necessário abrir o arquivo borutapy e retirar todos os np.int, np.float, np.bool e converter para int, float e bool, sucessivamente.

In [1]:
'''!pip install pandas
!pip install sklearn
!pip install boruta
!pip install pgmpy
!pip install networkx
!pip install matplotlib'''

'!pip install pandas\n!pip install sklearn\n!pip install boruta\n!pip install pgmpy\n!pip install networkx\n!pip install matplotlib'

In [2]:
'''!pip install --upgrade --force-reinstall pgmpy'''


'!pip install --upgrade --force-reinstall pgmpy'

In [7]:
#Método para realizar o Boruta, e ao final, montar um DataFrame com a ordem selecionada pelo boruta

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy
from sklearn.preprocessing import LabelEncoder

def boruta_feature_order(data_path, target_column):
    # Carregar os dados do CSV
    D = pd.read_csv(data_path)
    variables = D.columns
    print(f'Lista de variáveis: {variables}')

    # Print para verificar os dados antes do processamento
    print("Dados antes do processamento:")
    print(D)
    tipo_antes = (type(D[target_column][0]))
    print(f'Tipo dos dados antes do processamento: {tipo_antes}')
    

    # Mapear os valores nominais para números inteiros únicos
    D_encoded = D.apply(LabelEncoder().fit_transform)
    # Print para verificar os dados após o processamento
    print("Dados após o processamento:")
    print(D_encoded)
    tipo_depois = (type(D_encoded[target_column][0]))
    print(f'Tipo dos dados após o processamento (inteiros): {tipo_depois}')

    # Separar os dados em características (X) e alvo (y)
    X = D_encoded.drop(columns=[target_column])
    y = D_encoded[target_column]

    # Inicializar um classificador Random Forest
    rf = RandomForestClassifier(n_estimators=100, n_jobs=-1)

    # Inicializar o Boruta
    boruta_selector = BorutaPy(rf, n_estimators='auto', verbose=2)

    # Ajustar o Boruta aos dados
    boruta_selector.fit(X.values, y.values)
    print("Boruta inicializado com sucesso")

    # Obter as características selecionadas
    print("Variáveis na ordem inicial")
    print(variables)
    
    selected_features = X.columns[boruta_selector.support_]
    print("Features selecionadas")
    print(selected_features)
    # Obter as características não selecionadas
    unselected_features = X.columns[~boruta_selector.support_]
    print("Features não selecionadas")
    print(unselected_features)

    # Combinar características selecionadas e não selecionadas, ordenando-as pela ordem de ranking do Boruta
    all_features = [target_column] + list(selected_features) + list(unselected_features)
    print("Todas as features, incluindo a target: ")
    print(all_features)

    # Reorganizar o DataFrame original de acordo com a ordem das características selecionadas pelo Boruta
    df_reordered = D[all_features]
    
    # Salvar o DataFrame reordenado no formato CSV
    df_reordered.to_csv("data_reordered.csv", index=False)

    print("DataFrame reordenado salvo como data_reordered.csv")
    print("DataFrame criado na ordem das features selecionadas pelo Boruta")
    print(df_reordered)
    
    return all_features

# Caminho para o arquivo CSV e o nome da coluna alvo
data_path = "contact-lenses.csv"
target_column = "astigmatism"

# Obter todas as características, incluindo a coluna alvo
all_features= boruta_feature_order(data_path, target_column)
print("Features em ordem")
print(all_features)

Lista de variáveis: Index(['age', 'spectacle-prescrip', 'astigmatism', 'tear-prod-rate',
       'contact-lenses'],
      dtype='object')
Dados antes do processamento:
               age spectacle-prescrip astigmatism tear-prod-rate  \
0            young              myope          no        reduced   
1            young              myope          no         normal   
2            young              myope         yes        reduced   
3            young              myope         yes         normal   
4            young       hypermetrope          no        reduced   
5            young       hypermetrope          no         normal   
6            young       hypermetrope         yes        reduced   
7            young       hypermetrope         yes         normal   
8   pre-presbyopic              myope          no        reduced   
9   pre-presbyopic              myope          no         normal   
10  pre-presbyopic              myope         yes        reduced   
11  pre-presbyopi

In [8]:
from pgmpy.models import BayesianNetwork, BayesianModel
from pgmpy.estimators import K2Score, BayesianEstimator
import pandas as pd
import random

def k2(dataset, parents_nmax):
    variables = list(dataset.columns)
    print(f'Ordem das variáveis iniciais: {variables}')
    estimator = K2Score(dataset)
    model = BayesianNetwork()
    nodes = list(variables)  # nome dos nos
    model.add_nodes_from(nodes)

    for i in range(1, len(nodes)):
        node = nodes[i]  # no atual
        previous_nodes = nodes[:i]  # nos anteriores ao no atual
        parents = []  # pais do no atual
        P_old = estimator.local_score(node, parents)
        proceed = True

        while proceed and (len(parents) < parents_nmax):
            candidates = list(set(previous_nodes) - set(parents))  # nos candidatos para pais do no atual
            P_new = P_old  # new probability

            for candidate in candidates:
                candidate_score = estimator.local_score(node, parents + [candidate])  # pontuacao dos nos pais

                if candidate_score > P_new:
                    candidates_best = candidate  # melhor candidato para no pai
                    P_new = candidate_score

            if P_new > P_old:
                P_old = P_new
                parents.append(candidates_best)
                model.add_edge(candidates_best, node)

            else:
                proceed = False
    md = {'Estrutura do Modelo': list(model.edges)}
    score = {'Score': estimator.score(model)}
    return md, model, score

# Função para encontrar as CPDs
def tabular_cpd(model, data):
    for column in data.columns:
        data[column] = pd.Categorical(data[column])
    estimator = BayesianEstimator(model, data)
    cpds = [estimator.estimate_cpd(node) for node in model.nodes]
    return cpds

data = pd.read_csv('data_reordered.csv')
best_model = k2(data, 4)
cpds = tabular_cpd(best_model[1], data)

print(f'Modelo: {best_model}')


Ordem das variáveis iniciais: ['astigmatism', 'contact-lenses', 'age', 'spectacle-prescrip', 'tear-prod-rate']
Modelo: ({'Estrutura do Modelo': [('astigmatism', 'contact-lenses'), ('contact-lenses', 'tear-prod-rate')]}, <pgmpy.models.BayesianNetwork.BayesianNetwork object at 0x000001B62D593B90>, {'Score': -99.0080200257895})


In [9]:
from pgmpy.readwrite import XMLBIFWriter

# Especifique o caminho do arquivo onde deseja salvar o arquivo XMLBIF
file_path = "teste_contact-lenses.xmlbif"

# Adicione as CPDs ao modelo
for cpd in cpds:
    best_model[1].add_cpds(cpd)

# Escreva o modelo no formato XMLBIF
writer = XMLBIFWriter(best_model[1]).write_xmlbif(file_path)

print(f"O arquivo XMLBIF foi gerado com sucesso em: {file_path}")


O arquivo XMLBIF foi gerado com sucesso em: teste_contact-lenses.xmlbif


TESTE:

1. Rodar 20x o K2 com ordem aleatória, para cada base de dados, e no fim tirar a média da função G (score), e o tempo;
    1.1 Guardar só a média das 20x;

2. Deixar anotado em uma planilha;

3. Rodar o K2 com o Boruta, para cada variável sendo classe, e a execução com o melhor G, eu guardo, guardando o tempo;

4. Guardar o XML;